전체 뉴스데이터를 하나의 데이터셋으로 Merging 

In [2]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

from konlpy.tag import Komoran 

from tqdm import tqdm

Total Data 합치기

In [3]:
# office_data=[['중앙일보','025'],['JTBC','437'],['YTN','052'],['한국경제','015'],['매일경제','009'],
#              ['조선일보','023'],['연합뉴스','422'],['SBS','055'],['KBS','056'],['한겨레','028']]

office_data=[['매일경제','009'],['머니투데이','008'],['비즈니스워치','648'],['서울경제','011'],
             ['아시아경제','277'],['이데일리','018'],['조선비즈','366'],['조세일보','123'],['파이낸셜뉴스','014'],['한국경제','015'],
            ['헤럴드경제','016']]
office_list=pd.DataFrame(office_data,columns=['언론사명','id'])

date_list=pd.read_excel('D:\\KMI-Project_tmp\\Data\\Sentimental\\Naver\\General\\DateList.xlsx')
df_content=pd.DataFrame()

In [4]:
def corpus(tmp_list):
    return ' '+' '.join(tmp_list)

In [5]:
def cleansing(text):  
    new_text=' '.join(text.split()) #공백 제거
    pattern = '[0-9]'
    new_text = re.sub(pattern=pattern,repl=' ',string=new_text)
    
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' #이메일 주소 제거
    new_text = re.sub(pattern=pattern, repl='', string=text)
    
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:\da-fA-F]{2}))+'  # url 제거
    new_text = re.sub(pattern=pattern,repl=' ',string=new_text)

    pattern = '<[^>]*>'                                          # html tag 제거
    new_text = re.sub(pattern=pattern,repl=' ',string=new_text)

    pattern = '[\r|\n]'                                          # \r, \n 제거
    new_text = re.sub(pattern=pattern,repl=' ',string=new_text)

    pattern = '[^\w\s]'                                          # 특수기호 제거
    new_text = re.sub(pattern=pattern,repl=' ',string=new_text)

    pattern = re.compile(r'\s+')                                 # 이중 space 제거
    new_text = re.sub(pattern=pattern,repl=' ',string=new_text)
    
    
    ##불용어 처리
    stopword="\d|급$|씨$|월$|일$|더불어민주당|새누리당|야당|여당|사진|이날|_|기자$|위|등|안|때|시|분|각" 

    word_tokens=word_tokenize(new_text)
    data_word=[x for x in word_tokens if not re.search(stopword,x)]
    new_text=" ".join(data_word)
    

    return(new_text)

In [6]:
#일반 뉴스 기사에 대한 중복, null 값 제거 과정 
df_total=pd.DataFrame()
list_error=[]
total_doc=0
komo=Komoran()
for office_id in office_list['id']:
    office_name=office_list.loc[office_list['id']==office_id,['언론사명']].values[0] #언론사명 
    office_name=office_name[0]
    print(office_name)
    
    for date in tqdm(date_list['Date']):
        file_name='Sentimental_Index_'+office_name+'_'+str(date)+'.xlsx'
        Docs=pd.read_excel('D:\\KMI-Project_tmp\\Data\\Sentimental\\Naver\\Economics\\A.전체\\'+file_name)

        for index, row in Docs.iterrows():
            content=row['Contents']; label=row['Label']; url=row['URL']
            content=cleansing(content) #string 형으로
            keyword=corpus(komo.nouns(content)) # string 형으로

            df_total=df_total.append(pd.Series([keyword,label,content,url],index=['Keyword','Label','Content','URL']),ignore_index=True) #전체 문서 파일 저장 

매일경제


100%|████████████████████████████████████████████████████████████████████████████████| 268/268 [09:34<00:00,  2.14s/it]


머니투데이


100%|████████████████████████████████████████████████████████████████████████████████| 268/268 [10:32<00:00,  2.36s/it]


비즈니스워치


...

100%|████████████████████████████████████████████████████████████████████████████████| 268/268 [11:56<00:00,  2.67s/it]


한국경제


100%|████████████████████████████████████████████████████████████████████████████████| 268/268 [15:07<00:00,  3.39s/it]


헤럴드경제


100%|████████████████████████████████████████████████████████████████████████████████| 268/268 [17:29<00:00,  3.92s/it]


In [7]:
df_total.dropna(how='any',inplace=True) #Null 값이 존재하는 행 제거 
df_total.drop_duplicates(subset=['Content'], inplace=True) #Contents 열에서 중복인 내용이 있다면 중복 제거 
df_total.to_excel('D:\\KMI-Project_tmp\\Data\\Sentimental\\Naver\\Economics\\TotalDoc_Economics.xlsx')

In [8]:
print('전체 뉴스 기사 수: ',len(df_total)); 

전체 뉴스 기사 수:  67190
